In [1]:
print("Nguyễn Nguyên Huy - 6351071030")

Nguyễn Nguyên Huy - 6351071030


In [6]:
#import thư viện
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [3]:
iris_dataset = load_iris()
X = iris_dataset.data
y = iris_dataset.target

print("--- THÔNG TIN DỮ LIỆU GỐC ---")
print(f"Kích thước biến X: {X.shape}")
print(f"Kích thước biến y: {y.shape}")
print(f"5 dòng đầu tiên của X (chưa chuẩn hóa):\n{X[:5]}")

scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

print("\n--- DỮ LIỆU SAU KHI CHUẨN HÓA [0, 1] ---")
print(f"Giá trị lớn nhất (Max): {X_normalized.max()}")
print(f"Giá trị nhỏ nhất (Min): {X_normalized.min()}")
print(f"5 dòng đầu tiên của X (đã chuẩn hóa):\n{X_normalized[:5]}")

--- THÔNG TIN DỮ LIỆU GỐC ---
Kích thước biến X: (150, 4)
Kích thước biến y: (150,)
5 dòng đầu tiên của X (chưa chuẩn hóa):
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]

--- DỮ LIỆU SAU KHI CHUẨN HÓA [0, 1] ---
Giá trị lớn nhất (Max): 1.0
Giá trị nhỏ nhất (Min): 0.0
5 dòng đầu tiên của X (đã chuẩn hóa):
[[0.22222222 0.625      0.06779661 0.04166667]
 [0.16666667 0.41666667 0.06779661 0.04166667]
 [0.11111111 0.5        0.05084746 0.04166667]
 [0.08333333 0.45833333 0.08474576 0.04166667]
 [0.19444444 0.66666667 0.06779661 0.04166667]]


In [5]:
'''
c) Định	nghĩa	hàm	create_model,	bao	gồm:
- Model	là	Neural Network	tương	tự	như	câu	1.h
- Biên	 dịch	 mô	 hình	 sử	 dụng optimizer:	 adam;	 loss:
sparse_categorical_crossentropy;	metric:	accuracy
'''
def create_model():
    """
    Hàm khởi tạo mô hình Neural Network cho bộ dữ liệu Iris.
    Kiến trúc giống câu 1.h
    """
    model = Sequential()

    # Hidden Layer 1: 10 neurons, Relu, Input shape = 4 (4 thuộc tính hoa)
    model.add(Dense(10, activation='relu', input_shape=(4,)))

    # Hidden Layer 2: 20 neurons, Relu
    model.add(Dense(20, activation='relu'))

    # Output Layer: 3 neurons (3 lớp hoa), Softmax
    model.add(Dense(3, activation='softmax'))

    # Biên dịch mô hình
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# --- Kiểm tra thử hàm ---
# Tạo thử một model để xem kiến trúc có đúng không
temp_model = create_model()
print("Đã định nghĩa xong hàm create_model!")
temp_model.summary()

Đã định nghĩa xong hàm create_model!


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 10)             │            50 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 20)             │           220 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            63 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 333 (1.30 KB)

 Trainable params: 333 (1.30 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
'''
d) Huấn	luyện	mô	hình	bằng	k-fold	cross	validation	(với	k=10)
Trong	quá	trình	huấn	luyện	lưu	và	in	ra	kết	quả	accuracy,	precision,	recall,	f1-
score	của	từng	fold.	Lưu	lại	mô	hình	sau	mỗi	fold.
'''
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# Khởi tạo các danh sách để lưu kết quả
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
models = []

fold_no = 1

print("--- BẮT ĐẦU HUẤN LUYỆN K-FOLD (K=10) ---")

# Vòng lặp huấn luyện
for train_index, val_index in kfold.split(X_normalized):

    # Chia dữ liệu
    X_train, y_train = X_normalized[train_index], y[train_index]
    X_val, y_val = X_normalized[val_index], y[val_index]

    # Tạo model mới
    model = create_model()

    # Huấn luyện (Tắt verbose=0 để đỡ rối mắt vì chạy 10 lần)
    # Dùng epochs=100, batch_size=1 theo đúng tinh thần bài Iris câu 1.h
    model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=0)

    # Dự đoán trên tập Validation của fold này
    y_pred_probs = model.predict(X_val, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)

    # Tính toán các chỉ số
    acc = accuracy_score(y_val, y_pred)
    # average='macro': Tính chỉ số cho từng lớp rồi chia trung bình không trọng số
    prec = precision_score(y_val, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_val, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_val, y_pred, average='macro', zero_division=0)

    # Lưu kết quả
    accuracy_scores.append(acc)
    precision_scores.append(prec)
    recall_scores.append(rec)
    f1_scores.append(f1)
    models.append(model)

    # In ra màn hình
    print(f"Fold {fold_no}: Acc={acc:.4f} | Prec={prec:.4f} | Rec={rec:.4f} | F1={f1:.4f}")

    fold_no += 1

print("--- ĐÃ HOÀN THÀNH HUẤN LUYỆN ---")

--- BẮT ĐẦU HUẤN LUYỆN K-FOLD (K=10) ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Fold 1: Acc=0.9333 | Prec=0.9167 | Rec=0.9444 | F1=0.9221


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Fold 2: Acc=1.0000 | Prec=1.0000 | Rec=1.0000 | F1=1.0000


...

Fold 8: Acc=1.0000 | Prec=1.0000 | Rec=1.0000 | F1=1.0000


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Fold 9: Acc=1.0000 | Prec=1.0000 | Rec=1.0000 | F1=1.0000


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Fold 10: Acc=0.9333 | Prec=0.9524 | Rec=0.9524 | F1=0.9487
--- ĐÃ HOÀN THÀNH HUẤN LUYỆN ---


In [8]:
'''
e) In	ra	Average	Accuracy,	Average	Recall,	Average	Precision,	Average	F1-
score	của	mô	hình	sau	quá	trình	huấn	luyện với	k-fold	cross	validation
'''
avg_acc = np.mean(accuracy_scores)
avg_prec = np.mean(precision_scores)
avg_rec = np.mean(recall_scores)
avg_f1 = np.mean(f1_scores)

print("--- KẾT QUẢ ĐÁNH GIÁ TRUNG BÌNH (10-FOLD CROSS VALIDATION) ---")
print(f"Average Accuracy : {avg_acc:.4f}  (hay {avg_acc*100:.2f}%)")
print(f"Average Precision: {avg_prec:.4f}")
print(f"Average Recall   : {avg_rec:.4f}")
print(f"Average F1-score : {avg_f1:.4f}")

--- KẾT QUẢ ĐÁNH GIÁ TRUNG BÌNH (10-FOLD CROSS VALIDATION) ---
Average Accuracy : 0.9533  (hay 95.33%)
Average Precision: 0.9574
Average Recall   : 0.9569
Average F1-score : 0.9511


In [9]:
'''
f) In	ra	Average	Recall,	Average	Precision,	Average	F1-score	cho	từng	class
của	mô	hình	sau	quá	trình	huấn	luyện với	k-fold	cross	validation.
'''
class_metrics = {
    0: {'prec': [], 'rec': [], 'f1': []},
    1: {'prec': [], 'rec': [], 'f1': []},
    2: {'prec': [], 'rec': [], 'f1': []}
}

# Tên các loài hoa
class_names = iris_dataset.target_names # ['setosa', 'versicolor', 'virginica']

# Thiết lập K-Fold
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

print("Đang tính toán chỉ số chi tiết cho từng lớp...")

# Vòng lặp K-Fold
for train_index, val_index in kfold.split(X_normalized):
    # Chia dữ liệu
    X_train, y_train = X_normalized[train_index], y[train_index]
    X_val, y_val = X_normalized[val_index], y[val_index]

    # Tạo và huấn luyện model (create_model đã định nghĩa ở câu c)
    model = create_model()
    model.fit(X_train, y_train, epochs=50, batch_size=1, verbose=0) # Giảm epoch xuống 50 cho nhanh

    # Dự đoán
    y_pred_probs = model.predict(X_val, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)

    # --- QUAN TRỌNG: average=None ---
    # Trả về mảng 3 phần tử tương ứng 3 lớp. Ví dụ: [1.0, 0.9, 0.8]
    p_arr = precision_score(y_val, y_pred, average=None, zero_division=0)
    r_arr = recall_score(y_val, y_pred, average=None, zero_division=0)
    f_arr = f1_score(y_val, y_pred, average=None, zero_division=0)

    # Lưu kết quả vào từ điển
    # Do validation có thể thiếu 1 số class (hiếm gặp), ta cần try/except hoặc check len
    # Nhưng với Iris và K=10, thường sẽ đủ 3 class. Ta loop qua 3 lớp:
    for cls in range(3):
        # Kiểm tra xem lớp đó có tồn tại trong y_val không để tránh lỗi
        if cls in np.unique(y_val):
             # Lấy đúng chỉ số tương ứng (lớp 0, 1, 2)
             # Lưu ý: p_arr sẽ có độ dài bằng số lượng class xuất hiện trong y_val + y_pred
             # Để an toàn nhất, ta dùng labels=[0,1,2] trong hàm score
             pass

    # Cách an toàn hơn: Tính lại với labels cố định
    p_arr = precision_score(y_val, y_pred, labels=[0, 1, 2], average=None, zero_division=0)
    r_arr = recall_score(y_val, y_pred, labels=[0, 1, 2], average=None, zero_division=0)
    f_arr = f1_score(y_val, y_pred, labels=[0, 1, 2], average=None, zero_division=0)

    for i in range(3):
        class_metrics[i]['prec'].append(p_arr[i])
        class_metrics[i]['rec'].append(r_arr[i])
        class_metrics[i]['f1'].append(f_arr[i])

# Tính trung bình và In kết quả
print("\n" + "="*60)
print(f"{'CLASS (Loài hoa)':<15} | {'Avg Precision':<15} | {'Avg Recall':<15} | {'Avg F1-score':<15}")
print("="*60)

for i in range(3):
    # Tính trung bình các fold
    avg_p = np.mean(class_metrics[i]['prec'])
    avg_r = np.mean(class_metrics[i]['rec'])
    avg_f = np.mean(class_metrics[i]['f1'])

    name = class_names[i]
    print(f"{name:<15} | {avg_p:.4f}          | {avg_r:.4f}          | {avg_f:.4f}")

print("="*60)

Đang tính toán chỉ số chi tiết cho từng lớp...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/lo


CLASS (Loài hoa) | Avg Precision   | Avg Recall      | Avg F1-score   
setosa          | 1.0000          | 1.0000          | 1.0000
versicolor      | 0.9690          | 0.9000          | 0.9208
virginica       | 0.9100          | 0.9600          | 0.9242


In [10]:
'''
g) Sử	dụng	các	mô	hình	phân	lớp	có	F1-score	tốt	nhất	đã	huấn	luyện	ở	trên
cho	03	mẫu	dữ	liệu	mới	sau:
[6.2,	2.9,	4.3,	1.3]
[5.1,	3.5,	1.4,	0.2]
[7.3,	2.8,	6.4,	2.1]
'''
print("--- DỰ ĐOÁN MẪU MỚI (SỬ DỤNG BEST MODEL) ---")

# Tìm mô hình có F1-score cao nhất trong quá trình K-Fold
# f1_scores là danh sách đã lưu ở câu d
best_fold_idx = np.argmax(f1_scores)
best_model_iris = models[best_fold_idx]
best_f1 = f1_scores[best_fold_idx]

print(f"-> Đã chọn mô hình từ Fold thứ {best_fold_idx + 1} (F1-score: {best_f1:.4f})")

# Định nghĩa dữ liệu mới
new_data = np.array([
    [6.2, 2.9, 4.3, 1.3],
    [5.1, 3.5, 1.4, 0.2],
    [7.3, 2.8, 6.4, 2.1]
])

# QUAN TRỌNG: Chuẩn hóa dữ liệu mới
# Sử dụng lại biến 'scaler' đã fit ở câu b
# Dùng hàm .transform() chứ KHÔNG dùng .fit_transform()
new_data_norm = scaler.transform(new_data)

print(f"\nDữ liệu sau khi chuẩn hóa:\n{new_data_norm}")

# Dự đoán
# model trả về xác suất -> dùng argmax lấy nhãn
probs = best_model_iris.predict(new_data_norm, verbose=0)
predictions = np.argmax(probs, axis=1)

# In kết quả rõ ràng
class_names = iris_dataset.target_names # ['setosa', 'versicolor', 'virginica']

print("\n--- KẾT QUẢ DỰ ĐOÁN ---")
for i in range(len(new_data)):
    print(f"Mẫu {i+1} {new_data[i]} -> Dự đoán: {class_names[predictions[i]]} (Lớp {predictions[i]})")

--- DỰ ĐOÁN MẪU MỚI (SỬ DỤNG BEST MODEL) ---
-> Đã chọn mô hình từ Fold thứ 2 (F1-score: 1.0000)

Dữ liệu sau khi chuẩn hóa:
[[0.52777778 0.375      0.55932203 0.5       ]
 [0.22222222 0.625      0.06779661 0.04166667]
 [0.83333333 0.33333333 0.91525424 0.83333333]]

--- KẾT QUẢ DỰ ĐOÁN ---
Mẫu 1 [6.2 2.9 4.3 1.3] -> Dự đoán: versicolor (Lớp 1)
Mẫu 2 [5.1 3.5 1.4 0.2] -> Dự đoán: setosa (Lớp 0)
Mẫu 3 [7.3 2.8 6.4 2.1] -> Dự đoán: virginica (Lớp 2)
